In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,SimpleRNN, GRU,Activation,Flatten
from tensorflow.keras.layers import Conv1D,MaxPool1D, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (precision_score, recall_score,confusion_matrix,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

keras: 2.6.0


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Admin\Downloads\unknown_labelled_data.csv')

In [6]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label_BENIGN,Label_DDoS,Label_DoS Hulk,Label_PortScan,Label_Unknown
0,0,-0.646799,0.324411,-1.948333,0.204423,2.428566,-0.662668,2.051947,-0.472445,-0.008076,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
1,1,1.783219,0.430907,0.508829,0.114303,-0.421518,1.505849,-0.394452,-0.469837,-0.009562,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
2,2,-0.302869,-0.414284,0.159378,-0.558287,-0.421518,1.505849,-0.416902,-0.472439,-0.009562,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
3,3,-0.126277,-1.034865,0.769223,-1.040696,-0.421518,1.505849,-0.407956,-0.470938,-0.006590,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0
4,4,-0.708056,0.979845,-1.948333,-0.560275,2.462290,-0.662668,-0.401372,-0.472445,-0.011049,...,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,1,0,0,0,0


In [7]:
df = df.sample(frac = 1)

In [8]:
df.drop(df.columns[0] , inplace=True , axis=1)

In [9]:
df.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [10]:
LabelColumns = ['Label_BENIGN', 'Label_DDoS','Label_DoS Hulk', 'Label_PortScan', 'Label_Unknown']

In [11]:
y = df[LabelColumns]

In [12]:
y

,Label_BENIGN,Label_DDoS,Label_DoS Hulk,Label_PortScan,Label_Unknown
1076769,1,0,0,0,0
803175,1,0,0,0,0
1020932,1,0,0,0,0
1827201,0,0,0,1,0
249850,1,0,0,0,0
...,...,...,...,...,...
1523187,0,0,1,0,0
707436,1,0,0,0,0
939174,1,0,0,0,0
1044940,1,0,0,0,0


In [13]:
X = df.drop(LabelColumns , axis=1)

In [14]:
X

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
1076769,0.473919,0.362169,0.922166,0.081170,-0.421518,1.505849,-0.343474,-0.471067,-0.009562,-0.007983,...,-0.006451,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
803175,1.591614,0.371850,1.006471,0.081170,-0.421518,1.505849,-0.403229,-0.470716,-0.009562,-0.007983,...,-0.006451,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1020932,-0.981367,-0.411380,0.881331,-0.564251,-0.421518,1.505849,-0.415720,-0.471556,-0.011049,-0.009102,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
1827201,-0.892391,-0.964191,-0.293948,-0.680214,2.385755,-0.662668,-0.397490,-0.472447,-0.011049,-0.009102,...,-0.008194,0.002722,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
249850,0.066274,-0.407507,0.940796,-0.558287,-0.421518,1.505849,-0.406099,-0.472443,-0.009562,-0.007983,...,-0.006451,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523187,-0.856425,-0.216782,0.192263,0.115628,-0.419975,-0.662668,-0.413188,1.956520,0.000841,-0.003509,...,-0.006451,0.002729,-0.127109,-0.097503,-0.145913,-0.104499,2.705004,-0.121018,2.610676,2.749007
707436,1.584085,0.484155,1.049188,0.208399,-0.421518,1.505849,2.113390,-0.471462,-0.009562,-0.007983,...,-0.006451,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320
939174,-0.709232,-0.413316,-0.033648,0.378700,-0.399227,-0.662668,-0.384661,-0.310810,0.000841,-0.003509,...,-0.002965,0.002729,0.331674,-0.097503,0.151851,0.410265,-0.197144,-0.121018,-0.208829,-0.180915
1044940,1.590316,0.429939,0.480319,0.114303,-0.421518,1.505849,-0.339761,-0.472442,-0.009562,-0.007983,...,-0.006451,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.400720,-0.380320


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#  Convolutional Neural Networks

In [16]:
X_train.shape , y_train.shape

((1240540, 83), (1240540, 5))

In [17]:
X_train.shape , y_train.shape , X_test.shape

((1240540, 83), (1240540, 5), (611013, 83))

In [18]:
X_train = X_train.to_numpy().reshape(1240540, 83,1)
X_test = X_test.to_numpy().reshape(611013, 83,1)


In [19]:

model_cnn = Sequential()
model_cnn.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(83, 1)))
model_cnn.add(Conv1D(64, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation="relu"))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(5, activation="sigmoid"))

In [20]:
model_cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stoppoing

#FIT AND SAVE MODEL FOR FUTURE
model_cnn.fit(X_train, y_train, epochs=1000,validation_data=(X_test, y_test),callbacks=[es])
model_cnn.save("/content/drive/MyDrive/unknown_labelled_cnn.hdf5")

Epoch 1/1000
38767/38767 [==============================] - 459s 11ms/step - loss: 0.0040 - accuracy: 0.9977 - val_loss: 0.0032 - val_accuracy: 0.9995
Epoch 2/1000
38767/38767 [==============================] - 260s 7ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.0147 - val_accuracy: 0.9996
Epoch 3/1000
38767/38767 [==============================] - 261s 7ms/step - loss: 0.0044 - accuracy: 0.9994 - val_loss: 9.1998e-04 - val_accuracy: 0.9997
Epoch 4/1000
38767/38767 [==============================] - 260s 7ms/step - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 5/1000
38767/38767 [==============================] - 297s 8ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.0020 - val_accuracy: 0.9996
Epoch 6/1000
38767/38767 [==============================] - 259s 7ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 0.0059 - val_accuracy: 0.9996
Epoch 7/1000
38767/38767 [==============================] - 260s 7ms/step - loss: 0.0217 - accu

In [22]:
model_cnn.save("unknown_labelled_cnn.hdf5")

# Accuracies Measure && Model Power

In [21]:
y_pred_cnn = model_cnn.predict_classes(X_test)
y_probs_cnn=model_cnn.predict_proba(X_test)
np.savetxt('cnn_predictions.txt', np.transpose(np.concatenate((y_test.reshape((y_test.size, 1)),y_pred_cnn), axis=1)), fmt='%01d')
np.savetxt('cnn_prob_predictions.txt', np.around(np.transpose(y_probs_cnn),decimals=5), fmt='%.5f')


AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
accuracy = accuracy_score(y_test, y_pred_cnn)
print("accuracy:",accuracy)
f1score=f1_score(y_test, y_pred_cnn)
print("f1-acore:",f1score)
cm=confusion_matrix(y_test, y_pred_cnn)
print("confusion matrix:\n",cm)
pr=precision_score(y_test,y_pred_cnn)
print("Precision:",pr)
rs=recall_score(y_test,y_pred_cnn)
print("Recall_score:",rs)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred_cnn, labels=model_cnn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model_cnn.classes_)
plt.title("CNN-Confusion Matrix")
plt.show()

## ROC Curves

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
plt.figure()
fpr, tpr, thresholds =  roc_curve(y_test, y_probs_cnn)
auc =  roc_auc_score(y_test,y_pred_cnn)
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('CNN', auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('ROC -- Deep Learning Algorithms')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize']=(16,8)
plt.show()   # Display
